In [3]:
%load_ext autoreload

%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from utils import _DeepLiftShap, _GradCAMPlusPlus
from data import PascalVOC2007
from results.results_metrics import ResultMetrics
from models import vgg11_PascalVOC

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
torch.manual_seed(123)
np.random.seed(123)

Here the VGG-11 model is instantiated, the last layer is replaced with a new layer that has 20 outputs instead of 1000 (because PascalVOC has 20 classes, but the model was trained on ImageNet which has 1000 classes). The loaded weights are pretrained on ImageNet and the fine-tuned on PascalVOC. For more detail about the fine-tuning process, see `fine_tuning.py`.

In [4]:
model = vgg11_PascalVOC()
model.to(device)
# Load the pretrained weights
model.load_state_dict(torch.load('VGG11_PascalVOC.pt', map_location=device))
model.eval()

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),                 # Convert to Tensor
    transforms.Normalize(                  # Normalize using ImageNet mean and std
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])


In [5]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [6]:
import torchsummary

torchsummary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
              ReLU-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         590,080
             ReLU-10          [-1, 256, 56, 56]               0
        MaxPool2d-11          [-1, 256, 28, 28]               0
           Conv2d-12          [-1, 512, 28, 28]       1,180,160
             ReLU-13          [-1, 512, 28, 28]               0
           Conv2d-14          [-1, 512,

In [7]:
test_data = PascalVOC2007("test", transform=preprocess)
train_data = PascalVOC2007("trainval", transform=preprocess)

Using downloaded and verified file: data\VOCtest_06-Nov-2007.tar
Extracting data\VOCtest_06-Nov-2007.tar to data
Using downloaded and verified file: data\VOCtrainval_06-Nov-2007.tar
Extracting data\VOCtrainval_06-Nov-2007.tar to data


The train data is only used in the case of DeepLiftSHAP, because it requires a baseline distribution.

In [8]:
import torch.utils
from torch.utils.data import Subset

BATCH_SIZE_TEST = 1
NUM_TEST = 1024

BATCH_SIZE_TRAIN = 2
NUM_TRAIN = 8

dl_test = DataLoader(Subset(test_data, torch.randperm(len(test_data))[:NUM_TEST]), batch_size=BATCH_SIZE_TEST, shuffle=False)
dl_train = DataLoader(Subset(train_data, torch.randperm(len(train_data))[:NUM_TRAIN]), batch_size=BATCH_SIZE_TRAIN, shuffle=True)

The sum of all the attributions is equal to the difference between the output of the model on the current input, minus the average output of the model on the baseline distribution.

- The parameter `attribution_method` is an object that extends the base class `AttributionMethod` and implements the method `attribute`. In this way, simply changing the object passed to the `explain` method, we can change the attribution method used.
- The parameter `train_dl` is only used by the DeepLiftSHAP method, because it requires a baseline distribution.
- The function automatically add the metrics to the `ResultMetrics` object passed as parameter.
- The boolean `rescale_saliency` is used to rescale the saliency map in such a way that the integral of the saliency map is equal to $r \times (H \times W)$, where $H$ and $W$ are the height and width of the saliency map, and $r \in [0, 1]$ is the rescale factor.
    - This is useful to compare different saliency maps methods, because, for example, producing a saliency map with all ones is not useful, but gives the best results for some of the metrics.
    - The scaling is done in such a way that the positions and the values where the saliency map is equal to one are not changed. More formally, given the saliency map $S(i, j): \mathbb{R}^2 \rightarrow [0,1]$, the rescaled saliency map $S'(i, j) = S(i,j)^{\alpha^2}$ is found as an optimization problem by defining:

    $$L(S(i,j), \alpha) = \left( \sum_{i,j} S(i,j)^{\alpha^2}  - (r \times H \times W) \right)^2$$

    $$\alpha = \arg \min_{\alpha \in \mathbb{R}} L(S(i,j), \alpha)$$

    - It can be seen as a way to constraint the saliency map into keeping only a limited amount of important pixels.

In [7]:
len(model.features)

21

In [11]:
from metrics import calculate_metrics

RESULT_PATH = "./results.json"

results = ResultMetrics(RESULT_PATH)
calculate_metrics(model,
                _DeepLiftShap(),
                dl_test,
                dl_train,
                layers=[model.features[6], model.features[16]],
                result_metrics=results,
                device=device,
                upsample="bilinear",
                rescale_saliency=True,
                model_name="VGG11_Rescaling",
                debug=True
                )

  0%|          | 0/1024 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
Memory used: 2295.60 MB
Memory used: 3087.93 MB
Scaling took 0.22 seconds
Memory used: 3087.97 MB
Memory used: 2239.50 MB
Memory used: 2243.00 MB
Memory used: 2244.12 MB
Memory used: 2244.18 MB
--------------------------------------------------------------------------------
Memory used: 2244.18 MB
Memory used: 3079.07 MB
Scaling took 0.26 seconds
Memory used: 3079.18 MB
Memory used: 2339.58 MB
Memory used: 2341.58 MB
Memory used: 2341.30 MB
Memory used: 2341.29 MB
--------------------------------------------------------------------------------
Memory used: 2341.29 MB
Memory used: 3080.71 MB
Scaling took 0.34 seconds
Memory used: 3080.83 MB
Memory used: 2341.04 MB
Memory used: 2341.41 MB
Memory used: 2341.11 MB


KeyboardInterrupt: 

In [12]:
from metrics import calculate_metrics

RESULT_PATH = "./results.json"

results = ResultMetrics(RESULT_PATH)

for i in model.features:
    attr = _GradCAMPlusPlus(model, i)
    calculate_metrics(model,
                    attr,
                    dl_test,
                    dl_train,
                    layers=[i],
                    result_metrics=results,
                    device=device,
                    upsample="bilinear",
                    rescale_saliency=True,
                    model_name="VGG11_Rescaling")

  0%|          | 0/1024 [00:00<?, ?it/s]

Scaling took 0.28 seconds
Scaling took 0.20 seconds
Scaling took 0.27 seconds


KeyboardInterrupt: 